In [17]:
import os
path = 'datasets/NEU_DET/all_instances'
output_jsonpath = "datasets/NEU_DET/all_samples.json"
xml_list =[os.path.join(f"{path}/{xml_file_path}") for xml_file_path in os.listdir(path)]

label_path = 'datasets/NEU_DET/all_instances/label_path.txt'
with open(label_path,'w') as f:
    for xml_path in xml_list:
           f.write(xml_path)
           f.write('\n')



In [18]:
labels = ['crazing','inclusion','patches','pitted_surface','rolled-in_scale','scratches']
cats_path = 'datasets/NEU_DET/all_instances/labels.txt'
with open(cats_path,'w') as f:
    for label in labels:
       f.write(label)
       f.write('\n')



In [19]:
import os
import argparse
import json
import xml.etree.ElementTree as ET
from typing import Dict, List
from tqdm import tqdm
import re


def get_label2id(labels_path: str) -> Dict[str, int]:
    """id is 1 start"""
    with open(labels_path, 'r') as f:
        labels_str = f.read().split()
    labels_ids = list(range(1, len(labels_str)+1))
    return dict(zip(labels_str, labels_ids))


def get_annpaths(ann_dir_path: str = None,
                 ann_ids_path: str = None,
                 ext: str = '',
                 annpaths_list_path: str = None) -> List[str]:
    # If use annotation paths list
    if annpaths_list_path is not None:
        with open(annpaths_list_path, 'r') as f:
            ann_paths = f.read().split()
        return ann_paths

    # If use annotaion ids list
    ext_with_dot = '.' + ext if ext != '' else ''
    with open(ann_ids_path, 'r') as f:
        ann_ids = f.read().split()
    ann_paths = [os.path.join(ann_dir_path, aid+ext_with_dot) for aid in ann_ids]
    return ann_paths


def get_image_info(annotation_root, extract_num_from_imgid=True):
    path = annotation_root.findtext('path')
    if path is None:
        filename = annotation_root.findtext('filename')
    else:
        filename = os.path.basename(path)
    img_name = os.path.basename(filename)
    img_id = os.path.splitext(img_name)[0]
    if extract_num_from_imgid and isinstance(img_id, str):
        if img_id[:2] == 'cr':
            img_id = int(re.findall(r'\d+', img_id)[0]+'1111')
        elif img_id[:2] == 'in':
            img_id = int(re.findall(r'\d+', img_id)[0]+'2222')
        elif img_id[:2] == 'pa':
            img_id = int(re.findall(r'\d+', img_id)[0]+'3333')
        elif img_id[:2] == 'pi':
            img_id = int(re.findall(r'\d+', img_id)[0]+'4444')
        elif img_id[:2] == 'ro':
            img_id = int(re.findall(r'\d+', img_id)[0]+'5555')
        elif img_id[:2] == 'sc':
            img_id = int(re.findall(r'\d+', img_id)[0]+'6666')

    size = annotation_root.find('size')
    width = int(size.findtext('width'))
    height = int(size.findtext('height'))

    image_info = {
        'file_name': filename,
        'height': height,
        'width': width,
        'id': img_id
    }
    return image_info


def get_coco_annotation_from_obj(obj, label2id):
    label = obj.findtext('name')
    assert label in label2id, f"Error: {label} is not in label2id !"
    category_id = label2id[label]
    bndbox = obj.find('bndbox')
    xmin = int(bndbox.findtext('xmin')) - 1
    ymin = int(bndbox.findtext('ymin')) - 1
    xmax = int(bndbox.findtext('xmax'))
    ymax = int(bndbox.findtext('ymax'))
    assert xmax > xmin and ymax > ymin, f"Box size error !: (xmin, ymin, xmax, ymax): {xmin, ymin, xmax, ymax}"
    o_width = xmax - xmin
    o_height = ymax - ymin
    ann = {
        'area': o_width * o_height,
        'iscrowd': 0,
        'bbox': [xmin, ymin, o_width, o_height],
        'category_id': category_id,
        'ignore': 0,
        'segmentation': []  # This script is not for segmentation
    }
    return ann


def convert_xmls_to_cocojson(annotation_paths: List[str],
                             label2id: Dict[str, int],
                             output_jsonpath: str,
                             extract_num_from_imgid: bool = True):
    output_json_dict = {
        "images": [],
        "type": "instances",
        "annotations": [],
        "categories": []
    }
    bnd_id = 1  # START_BOUNDING_BOX_ID, TODO input as args ?
    print('Start converting !')
    for a_path in tqdm(annotation_paths):
        # Read annotation xml
        ann_tree = ET.parse(a_path)
        ann_root = ann_tree.getroot()

        img_info = get_image_info(annotation_root=ann_root,
                                  extract_num_from_imgid=extract_num_from_imgid)
        img_id = img_info['id']
        output_json_dict['images'].append(img_info)

        for obj in ann_root.findall('object'):
            ann = get_coco_annotation_from_obj(obj=obj, label2id=label2id)
            ann.update({'image_id': img_id, 'id': bnd_id})
            output_json_dict['annotations'].append(ann)
            bnd_id = bnd_id + 1

    for label, label_id in label2id.items():
        category_info = {'supercategory': 'none', 'id': label_id, 'name': label}
        output_json_dict['categories'].append(category_info)

    with open(output_jsonpath, 'w') as f:
        output_json = json.dumps(output_json_dict)
        f.write(output_json)

In [20]:
    # convert all xml files to coco json
    
    labels_path = cats_path
    ann_path_list = label_path
    
    label2id = get_label2id(labels_path=labels_path)
    ann_paths = get_annpaths(
        ext='xml',
        annpaths_list_path=ann_path_list
    )
    convert_xmls_to_cocojson(
        annotation_paths=ann_paths,
        label2id=label2id,
        output_jsonpath=output_jsonpath,
        extract_num_from_imgid=True
    )

Start converting !


100%|██████████| 1800/1800 [00:00<00:00, 12980.93it/s]


In [1]:
import json
f = json.load(open('/home/huemorgen/DefectDetection/datasets/NEU_DET/all_samples.json'))
f['annotations'][0].keys()

dict_keys(['area', 'iscrowd', 'bbox', 'category_id', 'ignore', 'segmentation', 'image_id', 'id'])

In [2]:
f['images'][0].keys()

dict_keys(['file_name', 'height', 'width', 'id'])